# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [0]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [2]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [0]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [0]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[ 1.     0.     0.     0.     0.     0.   ]
 [ 1.     1.     0.     0.     0.     0.   ]
 [ 1.     1.455  1.     0.     0.     0.   ]
 [ 1.     1.714  1.742  1.     0.     0.   ]
 [ 1.     1.882  2.276  2.039  1.     0.   ]
 [ 1.     2.     2.671  2.944  2.354  1.   ]] 

[[  3.000e+00   3.000e+00   3.000e+00   3.000e+00   3.000e+00   3.000e+00]
 [  0.000e+00  -1.125e+00  -1.636e+00  -1.929e+00  -2.118e+00  -2.250e+00]
 [  0.000e+00   0.000e+00   2.625e-01   4.574e-01   5.975e-01   7.013e-01]
 [  0.000e+00   2.220e-16   0.000e+00  -2.197e-02  -4.480e-02  -6.469e-02]
 [  0.000e+00  -4.528e-16   0.000e+00   6.939e-18   8.080e-04   1.902e-03]
 [  0.000e+00   4.123e-16   0.000e+00  -1.634e-17   0.000e+00  -1.585e-05]] 

[[  0.000e+00   0.000e+00   0.000e+00   0.000e+00   0.000e+00   0.000e+00]
 [  0.000e+00   0.000e+00   0.000e+00   0.000e+00   0.000e+00   0.000e+00]
 [  0.000e+00   0.000e+00   0.000e+00   2.220e-16  -1.110e-16  -1.665e-16]
 [  0.000e+00   0.000e+00   2.220e-16  -5.551e-17

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [0]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [0]:
np.linalg.matrix_rank(a1)

6

In [0]:
l, u = diy_lu(a1)

print(l, u)

[[ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]] [[ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]
 [ nan  nan  nan  nan  nan  nan]]


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [0]:
def check_minors(a):
  m = 0
  for N in range(a.shape[0] + 1):
    n = np.zeros((N, N), dtype=float)
    for k in range(N):
      for l in range(N):
        n[k,l]=a[k,l]
    if np.linalg.det(n) == 0:
      m += 1
      return False
  if m == 0:
    return True

In [12]:
check_minors(a)

True

In [15]:
check_minors(a1)

False

### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [0]:
def diy_lu_pivoting(a):
  N = a.shape[0]
  u = a.copy()
  L = np.eye(N)
  P = L
  for j in range(N-1):
    n = np.zeros((1, N-j))
    for i in range(N-j):
      n[0,i]=u[i+j,j]
    #row with max element
    row_number = np.argmax(abs(n)) + j

    p = np.eye(N)
    p[j], p[row_number] = p[row_number], p[j].copy()

    u=p@u
    P=p@P
    lam = np.eye(N)
    gamma = u[j+1:, j] / u[j, j]
    lam[j+1:, j] = -gamma
    u = lam @ u

    lam[j+1:, j] = gamma
    L = L @ lam
    return L, u, P


In [56]:
L, u, P = diy_lu_pivoting(a)
print(L)
print(u)

[[1. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0.]
 [1. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 1.]]
[[ 3.          3.          3.          3.          3.          3.        ]
 [ 0.         -1.125      -1.63636364 -1.92857143 -2.11764706 -2.25      ]
 [ 0.         -1.63636364 -2.11764706 -2.34782609 -2.48275862 -2.57142857]
 [ 0.         -1.92857143 -2.34782609 -2.53125    -2.63414634 -2.7       ]
 [ 0.         -2.11764706 -2.48275862 -2.63414634 -2.71698113 -2.76923077]
 [ 0.         -2.25       -2.57142857 -2.7        -2.76923077 -2.8125    ]]


Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices a and a1.

In [0]:
def get_a(L, u, P):
  return np.linalg.inv(P)@L@u

In [58]:
#check for matrix a
P, L, u = diy_lu_pivoting(a)
b = get_a(P,L,u)
print(b)
print(b-a)

[[3.         3.         3.         3.         3.         3.        ]
 [3.         1.875      1.36363636 1.07142857 0.88235294 0.75      ]
 [3.         1.36363636 0.88235294 0.65217391 0.51724138 0.42857143]
 [3.         1.07142857 0.65217391 0.46875    0.36585366 0.3       ]
 [3.         0.88235294 0.51724138 0.36585366 0.28301887 0.23076923]
 [3.         0.75       0.42857143 0.3        0.23076923 0.1875    ]]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.22044605e-16
  -1.11022302e-16 -1.66533454e-16]
 [ 0.00000000e+00  0.00000000e+00  2.22044605e-16 -5.55111512e-17
  -1.66533454e-16 -1.66533454e-16]
 [ 0.00000000e+00  0.00000000e+00 -1.11022302e-16 -1.66533454e-16
   1.66533454e-16  5.55111512e-17]
 [ 0.00000000e+00  0.00000000e+00 -1.66533454e-16 -1.66533454e-16
   5.55111512e

In [59]:
#check for matrix a1
P, L, u = diy_lu_pivoting(a1)
b = get_a(P,L,u)
print(b)
print(b-a1)

[[3.         3.         3.         3.         3.         3.        ]
 [3.         3.         1.36363636 1.07142857 0.88235294 0.75      ]
 [3.         1.36363636 0.88235294 0.65217391 0.51724138 0.42857143]
 [3.         1.07142857 0.65217391 0.46875    0.36585366 0.3       ]
 [3.         0.88235294 0.51724138 0.36585366 0.28301887 0.23076923]
 [3.         0.75       0.42857143 0.3        0.23076923 0.1875    ]]
[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  2.22044605e-16
  -1.11022302e-16 -1.66533454e-16]
 [ 0.00000000e+00  0.00000000e+00  2.22044605e-16 -5.55111512e-17
  -1.66533454e-16 -1.66533454e-16]
 [ 0.00000000e+00  0.00000000e+00 -1.11022302e-16 -1.66533454e-16
   1.66533454e-16  5.55111512e-17]
 [ 0.00000000e+00  0.00000000e+00 -1.66533454e-16 -1.66533454e-16
   5.55111512e